In [2]:
from datetime import datetime
from dataclasses import dataclass



class Connection:
    def __init__(self, line, departure_time, arrival_time, start_latitude, start_longitude, end_latitude, end_longitude, end_stop):
        self.line: str = line
        self.departure_time: datetime = departure_time
        self.arrival_time: datetime = arrival_time
        self.start_latitude: float = start_latitude
        self.start_longitude: float = start_longitude
        self.end_latitude: float = end_latitude
        self.end_longitude: float = end_longitude
        self.end_stop: str = end_stop
        
    def __repr__(self):
        return f"Connection({self.line}, {self.departure_time.strftime('%H:%M:%S')} -> {self.arrival_time.strftime('%H:%M:%S')}, {self.end_stop})"




class BusStop:
    def __init__(self, name: str):
        self.name: str = name
        self.connections: dict[str, list[Connection]] = {}

    def add_connection(self, end_stop: str, connection: Connection):
        if end_stop not in self.connections:
            self.connections[end_stop] = [connection]
        else:
            self.connections[end_stop].append(connection)
    
    def __repr__(self):
        return f"BusStop({self.name}, {len(self.connections)} connections)"



In [3]:
import pandas as pd
from datetime import datetime

datetime_format = '%Y-%m-%d %H:%M:%S'


def convert_time(time: str) -> datetime:
    hour = int(time[:2])
    if hour >= 24:
        hour -= 24
        return datetime.strptime(f'2025-01-02 {hour:02}{time[2:]}', datetime_format)
    else:
        return datetime.strptime(f'2025-01-01 {time}', datetime_format)


def add_connection(graph: dict[str, BusStop], start_stop: str,
                   end_stop: str, connection: Connection):
    if start_stop not in graph:
        graph[start_stop] = BusStop(name=start_stop)
    if end_stop not in graph:
        graph[end_stop] = BusStop(name=end_stop)
    graph[start_stop].add_connection(end_stop, connection)


def load_csv_data(filename: str):
    df = pd.read_csv(filename)
    graph: dict[str, BusStop] = {}

    for index, row in df.iterrows():
        start_stop = row['start_stop']
        end_stop = row['end_stop']
        departure_time = convert_time(row['departure_time'])
        arrival_time = convert_time(row['arrival_time'])
        start_lat = row['start_stop_lat']
        start_lon = row['start_stop_lon']
        end_lat = row['end_stop_lat']
        end_lon = row['end_stop_lon']
        connection = Connection(
            line=row['line'],
            departure_time=departure_time,
            arrival_time=arrival_time,
            start_latitude=start_lat,
            start_longitude=start_lon,
            end_latitude=end_lat,
            end_longitude=end_lon,
            end_stop=end_stop
        )
        add_connection(graph, start_stop, end_stop, connection)
    
    return graph

In [4]:
graph = load_csv_data('mpk_indexed.csv')

/var/folders/03/n179cc412h58x4qb9mt1bv_m0000gn/T/ipykernel_34872/4144027882.py:26: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [6]:
def find_earliest_connection(connections, current_time, previous_line=None, transfer_time=2):

    earliest = None
    earliest_departure = None
    
    for conn in connections:
        required_time = current_time
        if previous_line is not None and conn.line != previous_line:
            required_time = current_time + timedelta(minutes=transfer_time)
        
        if conn.departure_time >= required_time and (earliest is None or conn.departure_time < earliest_departure):
            earliest = conn
            earliest_departure = conn.departure_time
    
    return earliest

In [5]:
import heapq
from datetime import timedelta

In [11]:
def dijkstra(start_stop: str, end_stop: str, departure_time: str, graph: dict[str, BusStop], transfer_time: int = 2):

    if start_stop not in graph or end_stop not in graph:
        return None, []
    
    start_time = convert_time(departure_time)
    
    distances = {stop: float('inf') for stop in graph}
    previous = {stop: None for stop in graph}
    arrival_times = {stop: None for stop in graph}
    previous_lines = {stop: None for stop in graph}
    
    distances[start_stop] = 0
    arrival_times[start_stop] = start_time
    
    priority_queue = [(0, start_stop)]
    
    while priority_queue:
        current_distance, current_stop = heapq.heappop(priority_queue)
        
        if current_stop == end_stop:
            break
        
        if current_distance > distances[current_stop]:
            continue
        
        current_time = arrival_times[current_stop]
        current_line = previous_lines[current_stop]
        
        for next_stop, connections in graph[current_stop].connections.items():

            earliest_conn = find_earliest_connection(
                connections, 
                current_time, 
                previous_line=current_line, 
                transfer_time=transfer_time
            )
            
            if earliest_conn is None:
                continue
            
            travel_time = (earliest_conn.arrival_time - current_time).total_seconds() / 60
            
            if distances[current_stop] + travel_time < distances[next_stop]:
                distances[next_stop] = distances[current_stop] + travel_time
                previous[next_stop] = (current_stop, earliest_conn)
                arrival_times[next_stop] = earliest_conn.arrival_time
                previous_lines[next_stop] = earliest_conn.line
                
                heapq.heappush(priority_queue, (distances[next_stop], next_stop))
    
    if distances[end_stop] == float('inf'):
        return None, []
    
    path = []
    current = end_stop
    
    while current != start_stop:
        prev_stop, connection = previous[current]
        path.append(connection)
        current = prev_stop
    
    path.reverse()
    
    return distances[end_stop], path


def print_path(time_minutes, path):
    if path:
        print(f"Całkowity czas podróży: {time_minutes:.1f} minut")
        print("\nTrasa:")
        current_line = None
        for i, conn in enumerate(path, 1):
            transfer_info = ""
            if current_line is not None and conn.line != current_line:
                transfer_info = " (przesiadka)"
            
            print(f"{i}. Linia {conn.line}{transfer_info}: odjazd {conn.departure_time.strftime('%H:%M:%S')}, "
                  f"przyjazd {conn.arrival_time.strftime('%H:%M:%S')} -> {conn.end_stop}")
            
            current_line = conn.line
    else:
        print("Nie znaleziono trasy.")


def print_path(time_minutes, path):
    """Drukuje informacje o znalezionej ścieżce."""
    if path:
        print(f"Całkowity czas podróży: {time_minutes:.1f} minut")
        print("\nTrasa:")
        for i, conn in enumerate(path, 1):
            print(f"{i}. Linia {conn.line}: odjazd {conn.departure_time.strftime('%H:%M:%S')}, "
                  f"przyjazd {conn.arrival_time.strftime('%H:%M:%S')} -> {conn.end_stop}")
    else:
        print("Nie znaleziono trasy.")



start = "BROCHÓW"
end = "Psie Pole"
departure = "14:28:30"

total_time, route = dijkstra(start, end, departure, graph)

print_path(total_time, route)

Całkowity czas podróży: 47.5 minut

Trasa:
1. Linia 114: odjazd 14:29:00, przyjazd 14:30:00 -> Chińska
2. Linia 114: odjazd 14:30:00, przyjazd 14:32:00 -> BROCHÓW (Stacja kolejowa)
3. Linia 114: odjazd 14:32:00, przyjazd 14:33:00 -> Topolowa
4. Linia 114: odjazd 14:33:00, przyjazd 14:34:00 -> Wiaduktowa
5. Linia 114: odjazd 14:34:00, przyjazd 14:37:00 -> Karwińska
6. Linia 114: odjazd 14:37:00, przyjazd 14:38:00 -> Park Wschodni
7. Linia 114: odjazd 14:38:00, przyjazd 14:39:00 -> Armii Krajowej
8. Linia 114: odjazd 14:39:00, przyjazd 14:40:00 -> KRAKOWSKA (Centrum handlowe)
9. Linia 114: odjazd 14:40:00, przyjazd 14:41:00 -> Krakowska
10. Linia 114: odjazd 14:41:00, przyjazd 14:42:00 -> Na Niskich Łąkach
11. Linia 5: odjazd 14:44:00, przyjazd 14:46:00 -> pl. Zgody (Muzeum Etnograficzne)
12. Linia 5: odjazd 14:46:00, przyjazd 14:49:00 -> pl. Wróblewskiego
13. Linia 4: odjazd 14:52:00, przyjazd 14:55:00 -> Urząd Wojewódzki (Impart)
14. Linia 4: odjazd 14:55:00, przyjazd 14:57:00 -> most 